In [1]:
# In TensorFlow once you import the library, commands
# such as tf.constant create new nodes in a global
# "default graph". Then for example the tf.matmul
# command actually creates a new node whose value is
# tied by an equation to the values of c,d. When evaluated
# in a Session via sess.run, the values of these derived
# nodes can be extracted.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

from tensorflow.python.ops import array_ops

# Build a dataflow graph.
c = tf.constant([[1.0, 2.0], [3.0, 4.0]])
d = tf.constant([[1.0, 1.0], [0.0, 1.0]])
e = tf.matmul(c, d)
x = tf.constant([[1.0, 2.0, 3.0]])
t = tf.matmul(tf.transpose(x),x)
t = tf.reshape(t,[-1])

# Construct a `Session` to execute the graph.
sess = tf.Session()

# Execute the graph and store the value that `e` represents in `result`.
result = sess.run(t)
print(result)

[ 1.  2.  3.  2.  4.  6.  3.  6.  9.]


In [ ]:
# Here is an example of the main loop in our linear logic RNN
# the batch size is 2, input size is 1, state size is 2

inputs = tf.constant([[0.0],[1.0]])
state = tf.constant([[1.0, 2.0],[3.0, 4.0]])
h = array_ops.concat(1, [inputs,state])

# Now the row index in inputs and shape currently represents the batch
# we convert this to a list index using unpack, after first inserting
# a new index to be the row
h2 = tf.expand_dims(h,1)
h3 = tf.unpack(h2) # a list, containing 1x3 row vectors
li = []
for row in h3:
    li.append(tf.matmul(tf.transpose(row),row))
h4 = tf.pack(li)
h5 = tf.reshape(h4,[int(h4.get_shape()[0]),-1]) # convert back to a 2D tensor

print("inputs shape: " + str(inputs.get_shape()))
print("inputs is: ")
print(sess.run(inputs))
print("state shape: " + str(state.get_shape()))
print("state is: ")
print(sess.run(state))
print("h shape: " + str(h.get_shape()))
print("h is: ")
print(sess.run(h))
print("h2 shape: " + str(h2.get_shape()))
print("first element of h3 shape: " + str(h3[0].get_shape()))
print("first element of h3 is: ")
print(sess.run(h3[0]))
print("h4 shape: " + str(h4.get_shape()))
print(sess.run(h4))
print("h5 shape: " + str(h5.get_shape()))
print(sess.run(h5))

In [ ]:
# In practice however the situation is more complicated,
# because TF does not infer the batch size and hence cannot
# use tf.pack and tf.unpack. Instead we use TensorArray, as follows

batch_size = 2
TensorArr = tf.TensorArray(tf.float32, 1, dynamic_size=True, infer_shape=False)
h3 = TensorArr.unpack(h2)

print("h2 shape: " + str(h2.get_shape()))
print("h3 size: " + str(h3.size()))
print("h3 first element: ")
print(sess.run(h3.read(0)))

h3p = tf.TensorArray(tf.float32, 1, dynamic_size=True,infer_shape=False)
for row in range(batch_size):
    row_tensor = h3.read(row)
    
    # Note it's important to put h3p = h3p.write each time
    h3p = h3p.write(row, tf.matmul(tf.transpose(row_tensor),row_tensor))

h4 = h3p.pack()
h5 = tf.reshape(h4,[2,-1]) # convert back to a 2D tensor

print("h5 shape: " + str(h5.get_shape()))
print(sess.run(h5))

In [ ]:
# Alternatively we can use map_fn. Much better! Uses TensorArray under the hood
f = lambda x: tf.reshape(tf.matmul(tf.transpose(x),x),[-1])
h5 = tf.map_fn(f, h2)
print("h5 shape: " + str(h5.get_shape()))
print(sess.run(h5))

In [ ]:
# Trying to understand Tensor / np.array operations
c = tf.constant([[1.0, 2.0], [3.0, 4.0]])
b = tf.constant([-1.0, 2.0])
print(sess.run(c+b))

In [ ]:
def f_reverse(seq):
    t = [0]*len(seq)
    for j in range(len(seq)):
        t[len(seq)-j-1] = seq[j]
    return t

seq = [0,1,0,1,1]

f_reverse(seq)

In [ ]:
int(10.3)

In [ ]:
3**2

In [ ]:
c = tf.expand_dims(tf.constant([[1.0, 2.0], [3.0, 4.0]]),1)
print(sess.run(c))
print(c.get_shape())
d = tf.constant([[[1.0, 0.0], [0.0, 1.0]], [[2.0, 0.0], [0.0, 2.0]]])
e = tf.batch_matmul(c,d)
f = tf.squeeze(e)
print(sess.run(e))
print("")
print(sess.run(tf.reshape(e,[2,2])))

In [ ]:
operator_size = 2
state_size = 4

iota_matrix = []
for i in range(state_size):
    a = [0.0]*operator_size
    if i < operator_size:
        a[i] = 1.0
    iota_matrix.append(a)
    
rho_matrix = []
for i in range(operator_size):
    a = [0.0]*state_size
    if i < state_size:
        a[i] = 1.0
    rho_matrix.append(a)
    
print(iota_matrix)
print(rho_matrix)

In [ ]:
c = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
#d = tf.expand_dims(c,1)
make_diag = lambda A: tf.diag(A)
v = tf.map_fn(make_diag,c)
print(sess.run(v))
print(v.get_shape())
print(c.get_shape())
print(tf.diag([1.0,2.0,3.0]))

In [ ]:
# Set up the projections to and injections from H to H1 (see Remark 2.6)

rho_matrix = []
for i in range(state_size):
    a = [0.0]*operator_size
    if i < operator_size:
        a[i] = 1.0
    rho_matrix.append(a)
    
iota_matrix = []
for i in range(operator_size):
    a = [0.0]*state_size
    if i < state_size:
        a[i] = 1.0
    iota_matrix.append(a)
    
# If you're confused about the dimensions, it's because we are going
# to multiply on the RIGHT.

In [ ]:
def skiprepeat(seq):
    t = []
    for j in range(len(seq)):
        if j % 2 == 0:
            t.append(seq[j])
        else:
            t.append(seq[j-1])
    return t

print(skiprepeat([1,0,0,0]))

In [14]:
def rotation_tensor(size):
    """
    Returns rotation matrices as a [3,3,3] tensor, which is [R^0, R^1, ..., R^{size-1}] 
    where R is the rotation matrix sending the first basis element to the second and 
    the final basis element to the first.
    """
    one_hots = []
    for i in range(size):
        a = [0.0]*size
        a[i] = 1.0
        one_hots.append(tf.constant(a))

    R_list = []
    for i in range(size):
        R = []
        for j in range(size):
            index = (j + i) % size
            R.append(one_hots[index])
        R_list.append(tf.stack(R))

    R_tensor = tf.stack(R_list)

    return R_tensor

In [15]:
rotations = rotation_tensor(3)
print(rotations)

Tensor("stack_7:0", shape=(3, 3, 3), dtype=float32)


In [16]:
print(sess.run(rotations))

[[[ 1.  0.  0.]
  [ 0.  1.  0.]
  [ 0.  0.  1.]]

 [[ 0.  1.  0.]
  [ 0.  0.  1.]
  [ 1.  0.  0.]]

 [[ 0.  0.  1.]
  [ 1.  0.  0.]
  [ 0.  1.  0.]]]


In [ ]:
one_hots = []
size = 3
for i in range(size):
    a = [0.0]*size
    a[i] = 1.0
    one_hots.append(tf.constant(a))

In [ ]:
print(one_hots)

In [ ]:
a = tf.constant(np.arange(1, 13, dtype=np.int32),shape=[2, 2, 3]) 

In [ ]:
b = tf.constant(np.arange(13, 25, dtype=np.int32),shape=[2, 3, 2])

In [ ]:
c = tf.matmul(a, b)

In [ ]:
print(sess.run(c))

In [ ]:
print(c.get_shape()[0])

In [7]:
from numpy import array
import numpy as np
a = array([[[2.0,1.0],[3.0,4.0]],
           [[5.0,6.0],[7.0,8.0]]])
b = array([[1.0,2.0],
          [3.0,4.0]])
np.multiply(a,b)

array([[[  2.,   2.],
        [  9.,  16.]],

       [[  5.,  12.],
        [ 21.,  32.]]])

In [17]:
import learnfuncs

ImportError: No module named learnfuncs

In [28]:
def f_repetitionpattern(seq,pattern):
    t = []
    i = 0
    j = 0
    while(len(t) < len(seq)):
        t.append(seq[j % len(seq)])
        j = j + pattern[i % len(pattern)]
        i = i + 1
    return t 

In [30]:
pattern = [1,0,1,2,0,3,0,1,1,0,0,2,2,4,0,1,7,10,2,0,0]
func_to_learn = lambda s: f_repetitionpattern(s,pattern)

func_to_learn([1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1])

[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]